In [10]:
import numpy as np
import pandas as pd
import statsrat as sr
from statsrat.expr.predef.pvl_iti import *
from oc_renewal import oc_renewal
from spont_rec_explicit import spont_rec
from extn_rw_models import *
from plotnine import *

In [53]:
# This noteboook puts all Chapter 3 simulations together.
# Unlike the notebooks for individual simulations, this notebook
# does not include graphs.

# The "refined" model (with equal weights for elemental and configural features)
# is having trouble producing both AAB renewal and latent inhibition with the same parameter settings.

# The 'power_decay' model (which is the same as the "refined" model, except that it
# lacks configural features) has an easier time producing AAB renewal and latent
# inhibition with the same parameters.

#model = elem
model = power_decay
#model = refined
print(model.pars)

par_vals = {'elem': [0.2, 5],
            'power_decay': [0.001, 0.1, 2.0, 5.0],
            'refined': [0.01, 0.01, 2.0, 5.0]}

            min   max  default
drate       0.0   0.5     0.25
lrate_min   0.0   0.5     0.10
power       0.0   2.0     0.50
resp_scale  0.0  10.0     1.00


In [54]:
print('ABA renewal \n')

trials = aba_renewal.make_trials('experimental')
ds_exp = model.simulate(trials, par_vals[model.name])

trials = aba_renewal.make_trials('control')
ds_ctl = model.simulate(trials, par_vals[model.name])

print(aba_renewal.oats['renewal'].compute_total({'experimental': ds_exp, 'control': ds_ctl}))
print(aba_renewal.oats['renewal'].mean_resp({'experimental': ds_exp, 'control': ds_ctl}))

ABA renewal 

0.13170375575972537
       schedule     trial_name u_name  mean_resp
0  experimental  cs -> nothing     us   0.783291
0       control  cs -> nothing     us   0.651587


In [55]:
print('ABC renewal \n')

trials = abc_renewal.make_trials('experimental')
ds_exp = model.simulate(trials, par_vals[model.name])

trials = abc_renewal.make_trials('control')
ds_ctl = model.simulate(trials, par_vals[model.name])

print(abc_renewal.oats['renewal'].compute_total({'experimental': ds_exp, 'control': ds_ctl}))
print(abc_renewal.oats['renewal'].mean_resp({'experimental': ds_exp, 'control': ds_ctl}))

ABC renewal 

0.11384378399669204
       schedule     trial_name u_name  mean_resp
0  experimental  cs -> nothing     us   0.748916
0       control  cs -> nothing     us   0.635072


In [56]:
print('AAB renewal \n')

trials = aab_renewal.make_trials('experimental')
ds_exp = model.simulate(trials, par_vals[model.name])

trials = aab_renewal.make_trials('control')
ds_ctl = model.simulate(trials, par_vals[model.name])

print(aab_renewal.oats['renewal'].compute_total({'experimental': ds_exp, 'control': ds_ctl}))
print(aab_renewal.oats['renewal'].mean_resp({'experimental': ds_exp, 'control': ds_ctl}))

AAB renewal 

0.0821429949871163
       schedule     trial_name u_name  mean_resp
0  experimental  cs -> nothing     us   0.733730
0       control  cs -> nothing     us   0.651587


In [57]:
print('occasion setting renewal \n')

trials = oc_renewal.make_trials('different')
ds_diff = model.simulate(trials, par_vals[model.name])

trials = oc_renewal.make_trials('same')
ds_same = model.simulate(trials, par_vals[model.name])

print(oc_renewal.oats['renewal'].compute_total({'different': ds_diff, 'same': ds_same}))
print(oc_renewal.oats['renewal'].mean_resp({'different': ds_diff, 'same': ds_same}))

occasion setting renewal 

0.0027414319916234664
    schedule      trial_name u_name  mean_resp
0  different  cs1 -> nothing     us   0.517712
0       same  cs1 -> nothing     us   0.514971


In [58]:
print('spontaneous recovery \n')

ds_dict = {}
ds_dict[0] = model.simulate(spont_rec.make_trials('delay0'), par_vals[model.name])
ds_dict[50] = model.simulate(spont_rec.make_trials('delay50'), par_vals[model.name])
ds_dict[100] = model.simulate(spont_rec.make_trials('delay100'), par_vals[model.name])
ds_dict[200] = model.simulate(spont_rec.make_trials('delay200'), par_vals[model.name])
ds_dict[400] = model.simulate(spont_rec.make_trials('delay400'), par_vals[model.name])

df = pd.DataFrame({'delay': [0, 50, 100, 200, 400], 'train_response': 5*[0], 'test_response': 5*[0]})
df = df.set_index('delay', drop = True)
for k in ds_dict.keys():
    index_train = (ds_dict[k]['stage_name'] == 'pre-extn test') & (ds_dict[k]['t_name'] == 'main')
    df.loc[k, 'train_response'] = ds_dict[k]['b'].loc[{'t': index_train}].mean().values
    index_test = (ds_dict[k]['stage_name'] == 'spont rec test') & (ds_dict[k]['t_name'] == 'main')
    df.loc[k, 'test_response'] = ds_dict[k]['b'].loc[{'t': index_test}].max().values
df = df.reset_index()
df['test_pct'] = 100*df['test_response']/df['train_response']
print(df)

plot_b = (ggplot(df, aes(x = 'delay', y = 'test_pct')) + geom_line() + geom_point())
plot_b += theme_classic()
plot_b += ylab('% recovery (b)')

spontaneous recovery 

   delay  train_response  test_response   test_pct
0      0        0.950099       0.518019  54.522677
1     50        0.950099       0.533147  56.114869
2    100        0.950099       0.547480  57.623486
3    200        0.950099       0.573858  60.399805
4    400        0.950099       0.618271  65.074345


In [59]:
print('reinstatement \n')

trials = reinstatement.make_trials('experimental')
ds_exp = model.simulate(trials, par_vals[model.name])
ds_exp['schedule'] = 'extra shock'

trials = reinstatement.make_trials('control')
ds_ctl = model.simulate(trials, par_vals[model.name])
ds_ctl['schedule'] = 'no extra shock (control)'

print(reinstatement.oats['reinstatement'].compute_total({'experimental': ds_exp, 'control': ds_ctl}))
print(reinstatement.oats['reinstatement'].mean_resp({'experimental': ds_exp, 'control': ds_ctl}))

reinstatement 

0.1263147767702515
       schedule     trial_name u_name  mean_resp
0  experimental  cs -> nothing     us   0.780648
0       control  cs -> nothing     us   0.654333


In [60]:
print('latent inhibition \n')

trials = latent_inhib.make_trials('experimental')
ds_exp = model.simulate(trials, par_vals[model.name])

trials = latent_inhib.make_trials('control')
ds_ctl = model.simulate(trials, par_vals[model.name])

print(latent_inhib.oats['latent_inhibition'].compute_total({'experimental': ds_exp, 'control': ds_ctl}))
print(latent_inhib.oats['latent_inhibition'].mean_resp({'experimental': ds_exp, 'control': ds_ctl}))

latent inhibition 

0.042626809260577936
       schedule     trial_name u_name  mean_resp
0       control  cs -> nothing     us   0.860484
0  experimental  cs -> nothing     us   0.817857
